In [1]:
import numpy as np
import tensorflow as tf
import keras
from datasets import Dataset, load_dataset
import librosa
from keras import layers, models

print("Tensorflow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("Numpy version:", np.__version__)

/Users/petter/skole/master/courses/DAT255/project/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/petter/skole/master/courses/DAT255/project/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Tensorflow version: 2.16.2
Keras version: 3.8.0
Numpy version: 1.26.4


In [2]:
dataset = load_dataset("google/speech_commands", "v0.01")
# Overview of the dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 51093
    })
    validation: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 6799
    })
    test: Dataset({
        features: ['file', 'audio', 'label', 'is_unknown', 'speaker_id', 'utterance_id'],
        num_rows: 3081
    })
})


In [3]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

In [4]:
# Function to get a 10% subset
def get_subset(dataset_split, fraction=0.1):
    return dataset_split.train_test_split(test_size=1 - fraction)["train"]

# Creating 10% subsets
train_subset = get_subset(train_dataset)
validation_subset = get_subset(validation_dataset)
test_subset = get_subset(test_dataset)

In [5]:

# Check available devices
print("Available devices:")
for device in tf.config.list_physical_devices():
    print(device)

Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [14]:

def preprocess_audio(set):

    audio_array = set['audio']['array']
    sampling_rate = set['audio']['sampling_rate']

    mel_spectogram = librosa.feature.melspectrogram(y=audio_array, sr=sampling_rate, n_mels=128)

    log_mel_spectogram = librosa.power_to_db(mel_spectogram)

    log_mel_spectogram = np.expand_dims(log_mel_spectogram, axis=-1)

    return {'audio': log_mel_spectogram}

train_dataset = train_subset.map(preprocess_audio)
validation_dataset = validation_subset.map(preprocess_audio)
test_dataset = test_subset.map(preprocess_audio)

In [15]:
# A simple CNN model
model = models.Sequential([
    layers.InputLayer(shape=(128, 32, 1)),
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(12, activation='softmax') # Number of possible commands
])

# Compiling
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 126, 30, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 63, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 61, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 30, 6, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 11520)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       737,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │           780 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 756,940 (2.89 MB)

 Trainable params: 756,940 (2.89 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
def audio_generator(dataset):
    for sample in dataset:
        audio_features = sample['audio']
        label = sample['label']
        
        # Convert audio_features to a numpy array (if it's not already)
        audio_features = np.array(audio_features)
        
        # Ensure the audio features have the shape (128, 32, 1)
        # Pad or truncate if necessary (this assumes the audio data is 2D, with shape (128, n_features, 1))
        if audio_features.shape[1] < 32:
            # Pad the sequence if it's shorter than expected
            pad_width = 32 - audio_features.shape[1]
            audio_features = np.pad(audio_features, ((0, 0), (0, pad_width), (0, 0)), mode='constant')
        elif audio_features.shape[1] > 32:
            # Truncate the sequence if it's longer than expected
            audio_features = audio_features[:, :32, :]
        
        # Yield the audio features and label
        yield audio_features, label

def convert_to_tf_dataset(dataset):
    # Create a TensorFlow Dataset from the generator
    tf_dataset = tf.data.Dataset.from_generator(
        lambda: audio_generator(dataset), 
        output_signature=(
            tf.TensorSpec(shape=(128, 32, 1), dtype=tf.float32),  # The expected shape of the audio data
            tf.TensorSpec(shape=(), dtype=tf.int64)  # Adjust dtype according to your label type (e.g., tf.int64 for class labels)
        )
    )
    return tf_dataset


# Convert the train, validation, and test datasets
train_tf_dataset = convert_to_tf_dataset(train_dataset)
validation_tf_dataset = convert_to_tf_dataset(validation_dataset)
test_tf_dataset = convert_to_tf_dataset(test_dataset)


In [17]:
for sample in test_tf_dataset.take(1):
    print(sample)

(<tf.Tensor: shape=(128, 32, 1), dtype=float32, numpy=
array([[[-53.382347],
        [-53.382347],
        [-53.382347],
        ...,
        [-53.382347],
        [-53.382347],
        [-53.382347]],

       [[-53.382347],
        [-53.382347],
        [-53.382347],
        ...,
        [-53.382347],
        [-53.382347],
        [-53.382347]],

       [[-53.382347],
        [-53.382347],
        [-53.382347],
        ...,
        [-46.86158 ],
        [-53.382347],
        [-49.96153 ]],

       ...,

       [[-53.382347],
        [-53.382347],
        [-53.382347],
        ...,
        [-53.382347],
        [-53.382347],
        [-53.382347]],

       [[-53.382347],
        [-53.382347],
        [-53.382347],
        ...,
        [-53.382347],
        [-53.382347],
        [-53.382347]],

       [[-53.382347],
        [-53.382347],
        [-53.382347],
        ...,
        [-53.382347],
        [-53.382347],
        [-53.382347]]], dtype=float32)>, <tf.Tensor: shape=(), dtype=int64

2025-03-11 14:00:15.341279: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [18]:
print(model.input_shape)

(None, 128, 32, 1)


In [22]:
BATCH_SIZE = 128

train_tf_dataset = train_tf_dataset.batch(128).repeat().prefetch(tf.data.AUTOTUNE)
validation_tf_dataset = validation_tf_dataset.batch(128).repeat().prefetch(tf.data.AUTOTUNE)
test_tf_dataset = test_tf_dataset.batch(128).prefetch(tf.data.AUTOTUNE)

steps_per_epoch = len(train_tf_dataset) // 128  # 128 is the batch size
validation_steps = len(validation_tf_dataset) // 128


TypeError: The dataset is infinite.

In [ ]:
print("Fitting the model now!!!")

model.fit(
    train_tf_dataset.batch(128).repeat().prefetch(tf.data.AUTOTUNE),
    epochs=10,
    steps_per_epoch=steps_per_epoch,  # Explicitly set steps per epoch
    validation_data=validation_tf_dataset.batch(128).repeat().prefetch(tf.data.AUTOTUNE),
    validation_steps=validation_steps,  # Explicitly set validation steps
)

test_loss, test_accuracy = model.evaluate(test_tf_dataset)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

Fitting the model now!!!
Epoch 1/10
     40/Unknown 50s 1s/step - accuracy: 0.0366 - loss: 11.3608

2025-03-11 14:01:15.558247: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-11 14:01:15.558264: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 245012527435012120
2025-03-11 14:01:15.558269: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 7038651977633747270
2025-03-11 14:01:15.558274: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17991365502834336160
/Users/petter/skole/master/courses/DAT255/project/venv/lib/python3.9/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._

40/40 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.0366 - loss: 11.2261 - val_accuracy: 0.0442 - val_loss: 1.5450
Epoch 2/10


2025-03-11 14:01:22.125830: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-11 14:01:22.125849: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/Shape/_4]]
2025-03-11 14:01:22.125855: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2210037642486351988
2025-03-11 14:01:22.125859: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16032637927664470710
2025-03-11 14:01:22.125865: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17762629323097450032


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.0476 - loss: 1.9969

...

2025-03-11 14:09:37.442194: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-11 14:09:37.442218: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 12673140366469210316
2025-03-11 14:09:37.442223: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 7038651977633747270
2025-03-11 14:09:37.442228: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17991365502834336160
2025-03-11 14:09:37.442232: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10536680547213624366
2025-03-11 14:09:37.442235: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 15920895663665989054
2025-03-11 14:09:37.442240: I tensorflow/core/framework/local_rende

40/40 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.0306 - loss: 202453.8438 - val_accuracy: 0.0501 - val_loss: 311613.0625


2025-03-11 14:09:43.767306: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-11 14:09:43.767324: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/Shape/_4]]
2025-03-11 14:09:43.767331: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2210037642486351988
2025-03-11 14:09:43.767334: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16032637927664470710
2025-03-11 14:09:43.767342: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17762629323097450032


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 961ms/step - accuracy: 0.0855 - loss: 571146.1250
Test Loss: 558017.1875, Test Accuracy: 0.08116883039474487


2025-03-11 14:09:46.937240: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-03-11 14:09:46.937259: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/Shape/_4]]
2025-03-11 14:09:46.937267: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 2210037642486351988
2025-03-11 14:09:46.937271: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16032637927664470710
2025-03-11 14:09:46.937280: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 17762629323097450032
